In [1]:
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import importlib
from scipy.io import FortranFile

import time 
import cftime 

# import modules in other directories
sys.path.append('../Utils/')
import GridUtils as GrU
import MakePressures as MkP
import humiditycalcs as hum
import MyConstants as Con

sys.path.append('../Regridder/')
import GenRegrid as GR
import FVRegrid as FV


importlib.reload( GR )
importlib.reload( FV )


Using Flexible parallel/serial VertRegrid 
Using Flexible parallel/serial VertRegrid 


<module 'FVRegrid' from '/glade/work/juliob/PyRegridding/Drivers/../Regridder/FVRegrid.py'>

## START HERE FOR "BATCH"

In [3]:
RegridMethod="CONSERVE" 
foo = GR.prep(Dst='fv1x1',DstVgrid='L32',Src='ERA5',WOsrf=True, RegridMethod=RegridMethod )

In prep Src= ERA5 to Dst=fv1x1 
Generating regridding weights. Method CONSERVE : ESMF method= 2
 Src scripfile /glade/work/juliob/ERA5-proc/ERA5interp/grids/ERA5_640x1280_scrip.nc 
 Dst scripfile /glade/p/cesmdata/cseg/inputdata/share/scripgrids/fv0.9x1.25_141008.nc 
 Src topo file /glade/work/juliob/ERA5-proc/ERA5interp/phis/ERA5_phis.nc 
 Dst topo file /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1.25_nc3000_Nsw042_Nrs008_Co060_Fi001_ZR_160505.nc 
 L32 Dst vertical grid from /glade/p/cesmdata/cseg/inputdata/atm/cam/inic/se/f.e22.FC2010climo.ne30pg3_ne30pg3_mg17.cam6_2_022.002.cam.i.0020-01-01-00000_c200610.nc 
Prepping for ERA5 to fv1x1 proc in GenRegrid took  23.4934 seconds


In [4]:
#faa=GR.get_ERA5( year=2021,month=6,day=1,hour0=0 )

#faa=GR.get_ERA5( year=2017,month=1,day=1 , hour0=0)
faa=GR.get_ERA5( year=1980,month=1,day=15 , hour0=0)

Time tags for ERA5 files ...
198001
1980011500_1980011505
tzyx
Using DASK 
ERA5 Global mean surface pressure= 79287824129.61296
ERA5 Global mean surface pressure= 79285072391.0151
ERA5 Global mean surface pressure= 79282756903.65837
ERA5 Global mean surface pressure= 79279387701.76538
ERA5 Global mean surface pressure= 79275810441.58817
ERA5 Global mean surface pressure= 79273239915.04431
shape of a_model  (137,)
Reading one set of ERA5 vars took  113.4045 seconds


In [5]:
#pmid_ERA, pmid_CAM, pmid_CAM_zERA , te_ERA, te_ERA_xCAM , te_ERA_xzCAM  =GR.xRegrid()

ExitAfterWinds=False
ExitAfterTemperature=False
verstime=time.strftime("%Y%m%d_%H%M", time.gmtime() ) + '_'+RegridMethod  #+'_lnPS'
if ( ExitAfterTemperature == True):
    print( " Exiting after Temp " )
    pmid_ERA, lat_ERA, lon_ERA, te_ERA, \
    pmid_CAM_zERA, lat_CAM, lon_CAM, te_ERA_xCAM, \
    pmid_CAM, te_ERA_xzCAM, \
    ps_ERA, ps_CAM, ps_ERA_xCAM, \
    phis_ERA, phis_CAM, phis_ERA_xCAM = GR.xRegrid( ExitAfterTemperature=True, 
                                                    HorzInterpLnPs=True )
elif ( ExitAfterWinds == True):
    print( " Exiting after Winds " )
    pmid_ERA, lat_ERA, lon_ERA, u_ERA, \
    pmid_CAM_zERA, lat_CAM, lon_CAM, u_ERA_xCAM, \
    pmid_CAM, u_ERA_xzCAM, \
    ps_ERA, ps_CAM, ps_ERA_xCAM, \
    phis_ERA, phis_CAM, phis_ERA_xCAM = GR.xRegrid( ExitAfterWinds=True, 
                                                    HorzInterpLnPs=True )
else:
    print( " Running through and writing netcdf " )
    fii=GR.xRegrid(HorzInterpLnPs=False)
    fuu=GR.write_netcdf(version=verstime )

 Running through and writing netcdf 
starting xRegrid ERA5 _x_ fv1x1 at Wed Apr 26 13:34:51 2023 
Finished phis Horz Rgrd  0.3588 seconds
Finished ps Horz Rgrd  0.4172 seconds
Finished te_ERA Horz Rgrd  41.0337 seconds
amid_ERA     (137,)
ps_ERA_xCAM  (6, 192, 288)
te_ERA_xCAM  (6, 137, 192, 288)
Finding Te150 and P150 took  3.8727 seconds
 In PsAdjust 
PHIS (192, 288)
PHIS_CAM (192, 288)
PS (6, 192, 288)
Creating 3D P-fields etc., took   2.8460 seconds
WilliamsonOlson surface 
Williamson Olson surface took  6.2710 seconds
 going into vertical regrid of T 
Flexible VertRegrid using reshaped ARRAYS 
nworkers available  1
Using serial code which is faster for 1 worker
Pll'zd Vertical int 59.4719 seconds
 going into horz+vertical regrid of Q 
Horz RG in fullRegrid 
Vert RG in fullRegrid 
Flexible VertRegrid using reshaped ARRAYS 
nworkers available  1
Using serial code which is faster for 1 worker
Pll'zd Vertical int 25.2388 seconds
 ... Bottom filling took 3.5190 seconds
 going into horz

In [ ]:
import glob

##fuu=GR.write_netcdf(version='poo4' )
Nudge_Path=sorted(glob.glob('/glade/scratch/wchapman/inputdata/nudging/ERAI_fv09/L32/*.nc'))
DSe = xr.open_dataset(Nudge_Path[0],decode_times=False)
    

In [ ]:
lat = DSe['lat'].values
lon = DSe['lon'].values
slat = DSe['slat'].values
slon = DSe['slon'].values
ug = DSe['U'].values
vg = DSe['V'].values
us = DSe['US'].values
vs = DSe['VS'].values

In [ ]:
print("US --> ",DSe['US'].dims)
print("VS --> ",DSe['VS'].dims)



In [ ]:
US,VS,slat,slon = FV.uvStaggers(U=ug,V=vg,lon=lon,lat=lat)


In [ ]:
plt.plot( US[0,28,40,:] )
plt.plot( us[0,28,40,:] )


In [ ]:
print("FV U's. U:",np.shape(ug), " US:",np.shape(us))
print("FV V's. V:",np.shape(vg), " VS:",np.shape(vs))
nt,nz,ny,nx=np.shape(ug)
vgL=vg[0,28,:,:]
vsL=vs[0,28,:,:]
#vgLx = np.append( vgL , vgL[:,0].reshape(ny,1), axis=1)
#lonx = np.append( lon , lon[0].reshape(1), axis=0)

vgLx = np.concatenate( ( vgL[:,nx-1].reshape(ny,1), vgL) , axis=1)
lonx = np.concatenate( ( lon[nx-1].reshape(1) ,lon), axis=0)
lonx[0]=lonx[0]-360.

print(np.shape(vgL))
print(np.shape(vgLx))
print(lon[nx-1])

In [ ]:
print(lonx[0:2])
slon2=(lonx[1:]+lonx[0:-1] )/2.
plt.plot( slon - slon2 )

In [ ]:
from scipy.interpolate import RegularGridInterpolator as RGi

interp = RGi( (lat,lonx),  vgLx )

#bounds_error=False, fill_value=None)


Lat, SLon = np.meshgrid(lat, slon )

boo=interp( (Lat, SLon) ).T

In [ ]:

plt.plot(vsL[40,:])
plt.plot( boo[40,:] )

In [ ]:
file1='/glade/scratch/juliob/ERA5_x_fv1x1_L58_poo4_WO.2017-01-01-18000.nc'
dood=xr.open_dataset( file1 )

In [ ]:
print( np.shape(dood['U'] ) )
print( dood['TimeStamp'] )

In [ ]:
aa=np.zeros((5,30,30))
print(np.shape(aa))
print(np.shape(aa[0,:,:].reshape(1,30,30)))


In [ ]:
#pmid_ERA, pmid_CAM, pmid_CAM_zERA , te_ERA, te_ERA_xCAM , te_ERA_xzCAM  =GR.xRegrid()

pmid_ERA, lat_ERA, lon_ERA, te_ERA, \
pmid_CAM_zERA, lat_CAM, lon_CAM, te_ERA_xCAM, \
pmid_CAM, te_ERA_xzCAM, \
ps_ERA, ps_CAM, ps_ERA_xCAM, \
phis_ERA, phis_CAM, phis_ERA_xCAM = GR.xRegrid( ExitAfterTemperature=True, 
                                                HorzInterpLnPs=False, 
                                                Use_ps_ERA_xCAM_in_vert=True )


#fii=GR.xRegrid()

In [ ]:
clev=np.linspace(200,330,num=31)
dlev=np.linspace(-10,10,num=21)
#clev=np.linspace(277,287,num=31)
#ERA - 89 128
#CAM -124 72
print(lon_ERA[128], lat_ERA[89])

# Global
elon,wlon=0,360
slat,nlat=-90,90

# Tibet ...
#elon,wlon=70,100
#slat,nlat=25,40

# Andes ...
#elon,wlon=280,300
#slat,nlat=-40,-10

# Himalayan Front
#elon,wlon=85,95
#slat,nlat=25,28


kCAM=57
kERA=136
nCAM=5

fig,ax = plt.subplots(2,2,figsize=(18,18) ,squeeze=False )

i,j=0,0
cf = ax[i,j].contourf( lon_CAM, lat_CAM, te_ERA_xzCAM[nCAM,kCAM,:,:],levels=clev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
ax[i,j].set_title("ERA Temp on xzCAM grid from Python code")
plt.colorbar(cf,ax=ax[i,j] )

i,j=0,1
cf = ax[i,j].contourf( lon_CAM, lat_CAM, te_ERA_xCAM[nCAM,kERA,:,:],levels=clev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
ax[i,j].set_title("ERA Temp on xCAM/zERA grid")
plt.colorbar(cf,ax=ax[i,j] )


In [ ]:
# Let's look at some interensitng points here
#
location="Australia"
nCAM=3
if (location=="Himalayas"):
    #Himalayan peaks
    Plat,Plon = 36.,76.
    ylim=(60_000.,42_000.)
    xlim=(240,270 )


if (location=="Andes"):
    #Andean peaks
    Plat,Plon=-30.,290.
    ylim=(75_000.,60_000.)
    xlim=(280,300 )

if (location=="Australia"):
    #Australia desert (Alice Springs)
    Plat,Plon=-23.4,135.
    ylim=(96_000.,85_000.)
    xlim=(280,320 )

iCAM=np.argmin( np.abs( lon_CAM - Plon ))
jCAM=np.argmin( np.abs( lat_CAM - Plat ))
iERA=np.argmin( np.abs( lon_ERA - Plon ))
jERA=np.argmin( np.abs( lat_ERA - Plat ))

#print(te_ERA_xzCAM[nCAM,kCAM,ilat,ilon] )

plt.plot( te_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA Temp from Python code on xCAM and zERA' )

"""
plt.plot( te_ERA_xCAM_wo2[:, jCAM, iCAM]-0 , press_ERA[:, jCAM, iCAM], '-o' 
         , label='ERA Temp on xCAM and zERA from GUTS OF W&O' )
"""

plt.plot( te_ERA_xzCAM [nCAM,:, jCAM, iCAM], pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA Temp from Python code on xzCAM' )

plt.plot( te_ERA [nCAM,:, jERA, iERA] , pmid_ERA [nCAM,:, jERA, iERA], '-^' 
         , label='ERA Temp from nearby point ' )

"""
plt.plot( te_ERA_xzCAM_wo2[:, jCAM, iCAM]+2 , press_CAM[:, jCAM, iCAM], '-*' 
         , label='ERA Temp on xzCAM from GUTS OF W&O' )
"""

plt.ylim( ylim )
plt.xlim( xlim )


plt.legend(loc='upper left')


In [ ]:
print( np.minimum( [0.],[200])[0] )

In [ ]:
Dphis = phis_ERA_xCAM - phis_CAM
plt.plot(Dphis.flatten()-dphis_WO.flatten() )

In [ ]:
#fuu=GR.write_netcdf(version='test02-oldtopo')

## Check Mapping files

In [ ]:


#ERA - 89 128
#CAM -124 72
jCAM,iCAM = 124,72
jERA,iERA = 89,128
nCAM = 3

plt.plot( te_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA Temp from Python code on xCAM and zERA' )

plt.plot( te_ERA_xCAM_wo2[:, jCAM, iCAM] , press_ERA[:, jCAM, iCAM], '-o' 
         , label='ERA Temp on xCAM and zERA from GUTS OF W&O' )



#plt.plot( te_ERA_xzCAM [0,:,124, 72] , pmid_CAM [0,:,124, 72], '-o' , label='ERA Temp from Python code on xzCAM' )
#plt.plot( te_ERA[0,:,89, 128] ,pmid_ERA [0,:,89, 128] ,'-+', label='ERA Temp at closest xERA point' )


plt.ylim(88_000.,70_000.)
plt.xlim(270,290)
plt.legend()

print(ps_new[jCAM,iCAM])
print(ps_CAM[nCAM,jCAM,iCAM])

print(ps_old[jCAM,iCAM])
print(ps_ERA_xCAM[nCAM,jCAM,iCAM])

